Анализ датасета с ключевыми словами

In [49]:
import json
import os
from collections import Counter
from itertools import chain
from pathlib import Path
from typing import List
from zipfile import ZipFile

from keywords import Textrank

In [12]:
__path = '/data/churikov/headline_data/dataset/ru_kw_eval_datasets/data/'
dataset_path = Path(__path)

In [27]:
data: List[dict] = []
for z_name in os.listdir(dataset_path):
    with ZipFile(dataset_path / z_name) as z, z.open(z.namelist()[0]) as f:
        for str_obj in f:
            data.append(json.loads(str_obj.decode()))

In [28]:
len(data)

17266

In [29]:
data[0]

{'keywords': ['КОНСОЛИДАЦИЯ',
  'СИСТЕМАТИЗАЦИЯ ЗАКОНОДАТЕЛЬСТВА',
  'КОНСОЛИДИРУЮЩИЕ ЗАКОНОПРОЕКТЫ',
  'КОДИФИКАЦИЯ',
  'КОДЕКСЫ'],
 'title': 'К вопросу о консолидации как форме систематизации законодательства',
 'url': 'https://cyberleninka.ru/article/n/k-voprosu-o-konsolidatsii-kak-forme-sistematizatsii-zakonodatelstva',
 'abstract': 'Статья посвящена проблеме наличия такой формы , как , и описывает ее характеристики. Рассматриваются точки зрения правоведов относительно классификации форм систематизации нормативных правовых актов, выделяя место  в данных исследованиях. Изложены взгляды не только современных ученых, но и дается краткий обзор отечественной литературы дореволюционного и советского периодов. Приводятся примеры использования  в условиях правовых систем континентального права и в странах общего права.',
 'content': '16\nе\n\nнны\nво\nПраовреметва\nи с ударс\nгос\n\nТЕОРИЯ ПРАВА\nК вопросу о консолидации как форме систематизации\nзаконодательства\nРеутов Валерий Павлович, 

Нормализация текста

In [32]:
prep_data = []
for obj in data:
    obj['keywords'] = [k.lower() for k in obj['keywords']]
    obj['title'] = obj['title'].lower()
    if 'url' in obj:
        obj.pop('url') 
    obj['abstract'] = obj.get('abstract', '').lower().replace('\n', ' ')
    obj['content'] = obj['content'].lower().replace('\n', ' ')
    prep_data.append(obj)

In [33]:
len(prep_data)

17266

In [34]:
prep_data[0]

{'keywords': ['консолидация',
  'систематизация законодательства',
  'консолидирующие законопроекты',
  'кодификация',
  'кодексы'],
 'title': 'к вопросу о консолидации как форме систематизации законодательства',
 'abstract': 'статья посвящена проблеме наличия такой формы , как , и описывает ее характеристики. рассматриваются точки зрения правоведов относительно классификации форм систематизации нормативных правовых актов, выделяя место  в данных исследованиях. изложены взгляды не только современных ученых, но и дается краткий обзор отечественной литературы дореволюционного и советского периодов. приводятся примеры использования  в условиях правовых систем континентального права и в странах общего права.',
 'content': '16 е  нны во праовреметва и с ударс гос  теория права к вопросу о консолидации как форме систематизации законодательства реутов валерий павлович, доктор юридических наук, профессор, заведующий кафедрой теории и истории государства и права пермского государственного нацио

Найдем к-во ключевых слов

In [40]:
kw_counts = Counter(chain.from_iterable(kw['keywords'] for kw in prep_data))

In [48]:
print(len(kw_counts))
print(kw_counts.most_common(5))

29815
[('сша', 3257), ('россия', 2497), ('украина', 1807), ('спорт', 1806), ('в мире', 1444)]


textrank kw

In [51]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_fscore_support


def multilabel_classification_report(keywords_1: List[List[str]], keywords_2: List[List[str]]):
    ohe = OneHotEncoder(n_values=len(set(chain.from_iterable(keywords_1 + keywords_2))))
    m1, m2 = ohe.transform(keywords_1), ohe.transform(keywords_2)
    return precision_recall_fscore_support(m1, m2)

In [56]:
textrank = Textrank(words=5)
pred = [textrank.predict(t['content']) for t in prep_data[:10]]

In [57]:
pred

[['права', 'законодательства', 'для', 'консолидации как'],
 ['населения', 'населенных', 'территории', 'республики', 'кбр'],
 ['что', 'при', 'для', 'задачи', 'вектор'],
 ['человека', 'человек', 'как', 'его', 'индивидуальность'],
 ['возраста', 'больных', 'при', 'cancer', 'общая'],
 ['для', 'образования', 'образовательных', 'что', 'является'],
 ['она', 'как', 'для', 'пения', 'томске'],
 ['для', 'рис', 'покрытия', 'доступа при'],
 ['для', 'при', 'или', 'является', 'следов'],
 ['что', 'как', 'староверов', 'жизнь', 'жизни']]

In [58]:
[t['keywords'] for t in prep_data[:10]]

[['консолидация',
  'систематизация законодательства',
  'консолидирующие законопроекты',
  'кодификация',
  'кодексы'],
 ['кабардино-балкария',
  'горная зона',
  'активность экзогенных процессов',
  'антропогенное воздействие'],
 ['ряды фурье',
  'собственные функции оператора ротор',
  'уравнения навье-стокса',
  'задача коши',
  'глобальные решения',
  'системы галеркина',
  'пространства гильберта'],
 ['индивидуальность',
  'индивидуальное бытие человека',
  'уникальность',
  'неповторимость',
  'повседневность',
  'фактор',
  'общество',
  'индивидуальный стиль'],
 ['лучевая терапия',
  'рак шейки матки',
  'больные пожилого и старческого возраста'],
 ['высшее профессиональное образование',
  'качество подготовки кадров',
  'компетентностный подход',
  'конкурентоспособность',
  'рынок труда',
  'институциональные изменения',
  'результат образования'],
 ['русская вокальная школа',
  'культурная жизнь',
  'искусство',
  'педагогическая деятельность',
  'талантливые специалисты'],

In [ ]:
1+1